In [1]:
# Before running this project, you might want to turn on the colab gpu (Runtime>Change runtime type and select GPU as Hardware accelerator).
# Choose tensorflow 1.x
%tensorflow_version 1.x

TensorFlow 1.x selected.


Augmented Generative Chatbot
==============

In [2]:
# Install Newspaper3k, NLTK, GPT-2, NLPAug and Glove model
!pip install newspaper3k --quiet
!pip install nltk --quiet
!pip install gpt-2-simple --quiet
!pip install NLPAug --quiet
!wget https://github.com/allenai/spv2/raw/master/model/glove.6B.100d.txt.gz --quiet
!gzip -d /content/glove.6B.100d.txt.gz 

     |████████████████████████████████| 215kB 25.6MB/s 
     |████████████████████████████████| 92kB 12.3MB/s 
     |████████████████████████████████| 81kB 8.3MB/s 
     |████████████████████████████████| 7.4MB 25.5MB/s 
     |████████████████████████████████| 389kB 17.4MB/s 


In [3]:
import numpy as np 
import nltk 
import random
from random import randint
import re
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as naf
from nlpaug.util import Action
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
import nltk.data
import gpt_2_simple as gpt2
import newspaper
from newspaper import Article
import warnings
warnings.filterwarnings('ignore')

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
# Download different NLTK modules 
nltk.download ('punkt', quiet=True) 
nltk.download ('stopwords', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('wordnet', quiet=True)

# GPT2 Model options: 
gpt2.download_gpt2(model_name = "355M") 
#gpt2.download_gpt2(model_name = "124M") 
#gpt2.download_gpt2(model_name = "744M") 

Fetching checkpoint: 1.05Mit [00:00, 206Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 76.1Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 382Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:05, 270Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 284Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 165Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 200Mit/s]                                                       


In [7]:

# Adopted from https://www.geeksforgeeks.org/scraping-websites-with-newspaper3k-in-python/

# Defining a list of urls 
# If error is raised for the first article, please run this code again. 
#The error should non persist after 3-4 attempts. 
#If this procedure does not fix the error, download/load the file runing the next cell.

list_of_urls = ['https://www.tctmagazine.com/additive-manufacturing-3d-printing-news/interview-kadine-james-hobs-3d-printing-studio/', 
                'https://www.womenofwearables.com/new-blog/wow-woman-in-tech-and-3d-vr-and-ar-kadine-james-learner-maker-innovator-3d-technology-enthusiast',
                'https://www.fabbaloo.com/blog/2018/5/9/kadine-james-the-fourth-industrial-revolution-is-changing-the-way-we-receive-information-how-we-process-it-how-we-work-and-what-jobs-we-will-do-it-is-happening-at-an-unprecedented-pace'] 
  
  
  # Parse through each url and display its content 

for url in list_of_urls: 
    url_i = newspaper.Article(url="%s" % (url), language='en') 
    url_i.download() 
    url_i.parse() 
    url_i.nlp() 
    count = 0

    # Adopted from https://stackoverflow.com/questions/36571560/directing-print-output-to-a-txt-file
    
    print(url_i.text, file=open("output.txt", "a"))

    # Backup copy 
    print(url_i.text, file=open("output_backup.txt", "a"))


     



In [23]:
# Download output_backup.txt if previous cell does not work.
# Adapted from https://medium.com/@acpanjan/download-google-drive-files-using-wget-3c2c025a8b99
'''!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1F4fjhDPToGjvxTFeoEl441y8oeEk9O5S' -O output.txt
'''

# or load it from your device
'''from google.colab import files
files.upload()'''
# or, if this github has been cloned, the file should be here Examples_outputs/Output_backup/
# Make sure the file is placed into the /content/ colab folder, otherwise move it or change all paths.


--2021-01-10 15:51:53--  https://docs.google.com/uc?export=download&id=1F4fjhDPToGjvxTFeoEl441y8oeEk9O5S
Resolving docs.google.com (docs.google.com)... 172.217.15.78, 2607:f8b0:4004:810::200e
Connecting to docs.google.com (docs.google.com)|172.217.15.78|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-0o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/o4dnsljvh5n3gt18a2b02srqnoqcfskf/1610293875000/02428310132117897738/*/1F4fjhDPToGjvxTFeoEl441y8oeEk9O5S?e=download [following]
--2021-01-10 15:51:53--  https://doc-0k-0o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/o4dnsljvh5n3gt18a2b02srqnoqcfskf/1610293875000/02428310132117897738/*/1F4fjhDPToGjvxTFeoEl441y8oeEk9O5S?e=download
Resolving doc-0k-0o-docs.googleusercontent.com (doc-0k-0o-docs.googleusercontent.com)... 172.217.164.129, 2607:f8b0:4004:814::2001
Connecting to doc-0k-0o-docs.googleusercontent.com (doc-0k-0o-docs.google

'from google.colab import files\nfiles.upload()'

In [9]:
# Cleaning content, adopted from week 5.2
#load output.txt or output_backup.txt

df = open('/content/output.txt').read()
stopWords = set (stopwords.words('english'))

def clean_sentence(val):
  regex = re.compile('([^\s\w\.]|_)+')
  sentence = regex.sub('', val)#.lower()
  sentence = sentence.split(" ")

  for word in list (sentence):
    if word in stopWords:
      sentence.remove(word)

      sentence=" ".join(sentence)

      return sentence

df = clean_sentence(df)

print(df, file=open("output_clean.txt", "a"))

#?df

# This will tokenize our corpus, transforming it into a list

df = nltk.sent_tokenize(df) 

#create backup for later
df_backup = df.copy()

print(df, file=open("output_token.txt", "a")) 

In [7]:

# Shuffling words 
count = 0
while count < 5:
    
    random.shuffle(df)
    print(df, file=open("output_shuffled.txt", "a")) 
    count = count + 1 

df = open('/content/output_shuffled.txt').read()    


In [8]:
# Leveraging non-contextual embeddings (GloVe) to apply augmentation

# Adapted from https://neptune.ai/blog/data-augmentation-nlp

# Choose between shuffled corpus, clean corpus or original corpus.


#df=open('/content/output_backup.txt').read()
#df=open('/content/output_shuffled.txt').read()
df=open('/content/output_clean.txt').read()

aug_w2v = naw.WordEmbsAug(model_type='glove', model_path='/content/glove.6B.100d.txt',
    action="substitute")
aug_w2v.aug_p=0.2
augmented_text = aug_w2v.augment(df)
#This code append the result to our chosen corpus
#print(augmented_text, file=open("output_clean.txt", "a"))
#print(augmented_text, file=open("output_backup.txt", "a")) 
print(augmented_text, file=open("output_GloVe.txt", "a"))

In [17]:
#test on a small example (it can be run multiple times, the result will be appended on the .txt file)
df="Expand Hobs Studio at London's HereEast campus)"

aug_w2v = naw.WordEmbsAug(model_type='glove', model_path='/content/glove.6B.100d.txt',
    action="substitute")
aug_w2v.aug_p=0.2
augmented_text = aug_w2v.augment(df)
print(augmented_text, file=open("output_small_GloVe.txt", "a"))

In [10]:
#duplicating the tokenized corpus 

#get backup 
df = df_backup

count = 0
while count < 2:
    
    
    print(df, file=open("output_duplicated.txt", "a")) 
    count = count + 1 




In [12]:
# Load the chosen an option
df= 'output_duplicated.txt'
#df= 'output_backup.txt'
#df= 'output_shuffled.txt'
#df= 'output_clean.txt'


In [13]:
sess = gpt2.start_tf_sess()

In [14]:
# Adapted from https://minimaxir.com/2019/09/howto-gpt2/

gpt2.finetune(sess,
              dataset=df,
              model_name='355M',# Choose your favourite model, I run all my tests with the 355M model
              steps=50, #50 provides the best results
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=50,
              save_every=10
              )


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt


100%|██████████| 1/1 [00:00<00:00, 364.41it/s]

Loading dataset...
dataset has 9343 tokens
Training...


[10 | 23.81] loss=2.80 avg=2.80
Saving checkpoint/run1/model-10
[20 | 46.55] loss=1.24 avg=2.02
Saving checkpoint/run1/model-20
Instructions for updating:
Use standard file APIs to delete files with this prefix.
[30 | 84.16] loss=0.17 avg=1.39
Saving checkpoint/run1/model-30
[40 | 119.54] loss=0.13 avg=1.07
Saving checkpoint/run1/model-40
[50 | 154.99] loss=0.04 avg=0.86
Saving checkpoint/run1/model-50


In [20]:
while True:
  
  ques = input("Please ask for something, I will be glad to share my knowledge : ")  

# Choose between chatbot or 'dictionary'

  inp = ques+'[Kadine] :' #chatbot
  

  chatbot = gpt2.generate(sess,
                length=100,
                top_p=0.0,
                temperature = 0.6,  #higer is abstact, low is the same as input(between 0.6 and 1 works pretty well)
                include_prefix=False,
                prefix=inp,
                nsamples=1,
                batch_size=1,
                return_as_list=True
               
                )
  
  # We need a strig to apply .split function, which will truncate the chatbot when a dot is found. 
  chatbot = str (chatbot) 
  chatbot = (chatbot).split(".")[0]
  
   
  
  print(chatbot)
  
  print(chatbot, file=open("transcript.txt", "a"))
  





Please ask for something, I will be glad to share my knowledge : Come i social media a tuo parere influenzano i nuovi artisti?
['Come i social media a tuo parere influenzano i nuovi artisti?[Kadine] : no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no no']


KeyboardInterrupt: ignored